## Gradient boost

The main idea behind Gradient Boost is to create a strong predictive model using many "weak learners". Each "weak learner" is a predictive model that is not good on its own, but each successive "weak learner" learns through mistakes made by previous "weak learners". Each "weak learner" can be any type of a model, but the most common one people use are Decision Trees. So, we will focus solely on desicion trees as well.


General flow to Gradient Boost is as follows:

1. Create initial model $f_0$ whose guess is always the same for all data points.

2. For each data point in the training set $x^{(i)}$, calculate residual (error) $r^{(i)}$. (Though generally called pseudo-residual due to as scaling it a bit)

3. Build a "small" desision tree that tries to predict **errors** $r^{(i)}$ based on data points $x^{(i)}$.

4. Create the next model $f_1$, by adding scaled down version of a tree from part 3 to $f_0$. (This should remind you a bit of how gradient descent works. This is why this method is called Gradient Boost).

5. Repeat the process of finding new residuals, then building a new tree to predict them, and then adding scaled down version of that tree to $f_1$, and keep repeating this process until some stopping critiria is met.


This model can be used for regression and classification. Regression is a bit easier to understand, so we will start with that.

### Gradient Boost for Regression

First let's quickly go over the steps above to get the general flow, then we will add the math needed. Note though, the math will look a bit different form this general flow, but it will be actually the same.

1. To get initial model ($f_0$), we just take the average of targets as our prediction.

2. For each data point $x^{(i)}$ calculate the errors $r_1^{(i)}=y^{(i)}-f_0(x^{(i)})$ 

3. Build a tree model $T_1$ that predicts our residuals based on data points. We would usually limit how big the tree is to avoid overfit. We then modify each leaf to return a single value. If the leaf already have only one value, it will return that value, but if the leaf has more than one value, it will return the average of all values in that leaf.

4. We create a new model $f_1=f_0+\epsilon \cdot T_1$. The value $\epsilon$ is a learning rate. It scales down the tree which helps with avoiding overfitting again. The new model will have better prediction as we took a small step towards correct guesses.

5. We repeat the process, by getting new residuals $r_2$, building the tree model $T_2$ to guess new errors and then building a new model $f_2=f_1+\epsilon\cdot T_2$ and so on until we stop.


The usual stopping critiria is either reaching maximum number of trees (set as hyperparameter) and/or when the difference in residuals is too small. 

So after we stop, our final model is a sum of many weak models: $f_n=f_0+\epsilon\cdot T_1 + \epsilon\cdot T_2+ \dots \epsilon\cdot T_n$

Now, let's get to the math behind it:

We consider a loss function $L(x)=\frac{1}{2}(y-f(x))^2$, where $f(x)$ is some a prediction based on $x$. (this is just MSE divided by 2 for a single point). Note, I wrote this for a single point, MSE is basically the average of $L$ taking over all points.

1. Then we set our initial model to $f_0(x)=\mathop{\arg\min}_{\gamma} \sum_{i=1}^nL(y^{(i)}, \gamma)$. While this may look scary, it isn't that bad. All it says is that we are looking for a single value $\gamma$ for which our loss is minimized on the whole set. It is not hard to see that the $\gamma$ that gives us the minimum is just the average of all $y^{(i})$. 

2. Now we go through a loop creating residuals $r^{(i)}_{j}$ ($i$ goes through data points, $j$ is $j$-th iteration (or loop)). For simplicity of notation, I will skip $(i)$-th indecies and consider $r_j$ as collection of all residuals for all data points.

    Then the residuals are $$r_{j}=-\left.\frac{\partial L(y, f(x))}{\partial f(x)}\right|_{f(x)=f_{j-1}(x)}$$ This is just a residual (i.e. true minus predicted). We evaluated it by setting $f(x)=f_{j-1}(x)$. So, in the first loop is just $f_0$:

    $$r_1=y-f_0(x)=y-\gamma$$

    where $\gamma$ is the average of all $y$'s.

3. Now we build a tree $T_j$ to predict residuals. This tree will have some amount of leaves. Let's denote the set of all leaves in $j$-iteration by $I_j$ and each leaf in $I_j$ by $L_{jl}$). In each leaf $L_{jl}$ we calculate 'common' value:
   $$\gamma_{jl}=\mathop{\arg\min}_{\gamma} \sum_{x \in L_{jl}}L(y, f_{j-1}(x)+\gamma)$$

   This is similar to what we did to get $f_0(x)$, but this time we use "previous" guesses ($f_{j-1}(x)$) and we only go over values inside each leaf separatelly. But overall, it is still just the average of values in the leaf.

4. Now we create the new model. For given $x$,
   $$f_j(x)=f_{j-1}(x)+\epsilon \sum_{L_{ij}\in I_j}\gamma_{jl}I(x \in L_{jl})$$
   where $I(x \in L_{jl})=1$ if  $x\in L_{jl}$ and 0 otherwise. In other words, given $x$, $f_j(x)$ is the predicted value of old model plus the value from our tree $T_j$ scaled by some learning rate $\epsilon$.

### Gradient Boost for Clasification

To deal with clasification we use similar ideas to logistic regression. We will look at the binary classification case first:

1. To make our values numerical we will use log of odds (log-odds) followed by sigmoid to convert the values to probabilities, instead of the average. (Numerically, if we have 30 points in class 0 and 70 points in class 1, we would get $sigm(\ln(70/30))=sigm(0.847)=0.7$. Using normal threshold of 0.5, we basically classifying everyone to class 1 (i.e. majority class). What we are actually using here is negative log loss function as our loss function $L$.
   
2. Then we compute residuals which equal to the difference between true values (0 or 1) and trained value (0.7 in the example from part 1). This comes, as in regression case, from taking derivative of our loss function and evaluating at previous predictions. 

3. We build the tree to guess residuals. However, we cant just add residuals to previous log-odds and we can't really compute log-odds with a single value for example. So, we use the following formula for each leaf:

$$Output=\frac{\sum \text{Residuals in the leaf}}{\sum (\text{Previous Prob})(1-\text{Previous Prob})}$$

    Here the sums are taking over all values in the leaf. This formula comes from trying to minimize 
    
$$\gamma_{jl}=\mathop{\arg\min}_{\gamma} \sum_{x \in L_{jl}}L(y, f_{j-1}(x)+\gamma)$$

    but instead of trying to find derivative right away (it's hard), we approximate it using degree 2 Taylor polynomial, and taking derivative of it.

4. Then we update predictions by taking previous log-odds (i.e 0.847 in above example) and adding new outputs scaled by some learning rate. Then we convert it to probability by taking sigmoid function. This is our new model.

5. Then we repeat the process just like in the regression case.

### Multiclass classification

If we have $c$ classes, we one-hot encode them, and consider $c$ binary models for each class (a point belongs to a specific class or not). So we would use log-odds and sigmoid to get residuals. However, after sigmoid, we apply softmax to get actual classification probabilities that we compare to true labels. Then we will build $c$ trees predicting corresponding residuals and continue looking at it as if it is binary classification to get output values and update log-odds for corresponding location. We then can calculate cross-entropy loss to see the actual loss of the whole model. We repeat this process as many steps as needed.